In [1]:
from actor import *

Using backend: pytorch


In [2]:
from MARS.common.nn import masked_softmax
pred = torch.randn(size= (10,7))
mask = torch.randint(0,2, size = (10,7))

config = {
    'device': 'cpu',
    'n_atom_feat': 16,
    'n_node_hidden': 64,
    'n_bond_feat': 4,
    'n_edge_hidden': 128,
    'n_layers': 6,
    'vocab_size': 10,
    'batch_size': 128
}
actor = Actor(config)
with open('mol_test.txt', 'r') as f:
    mols = f.readlines()

mols = [Chem.MolFromSmiles(smi.split(',')[0]) for smi in mols if Chem.MolFromSmiles(smi.split(',')[0])]

graphs = [mol_to_dgl(mol) for mol in mols]
graphs = dgl.batch(graphs)


In [3]:
p_act, p_del, p_add, p_arm, p_global=actor.get_prob(graphs)

In [4]:
p_act

[tensor([0.5066, 0.4934], grad_fn=<SqueezeBackward1>),
 tensor([0.5064, 0.4936], grad_fn=<SqueezeBackward1>),
 tensor([0.5066, 0.4934], grad_fn=<SqueezeBackward1>),
 tensor([0.5065, 0.4935], grad_fn=<SqueezeBackward1>),
 tensor([0.5066, 0.4934], grad_fn=<SqueezeBackward1>),
 tensor([0.5064, 0.4936], grad_fn=<SqueezeBackward1>),
 tensor([0.5064, 0.4936], grad_fn=<SqueezeBackward1>),
 tensor([0.5065, 0.4935], grad_fn=<SqueezeBackward1>),
 tensor([0.5065, 0.4935], grad_fn=<SqueezeBackward1>),
 tensor([0.5067, 0.4933], grad_fn=<SqueezeBackward1>),
 tensor([0.5066, 0.4934], grad_fn=<SqueezeBackward1>),
 tensor([0.5068, 0.4932], grad_fn=<SqueezeBackward1>),
 tensor([0.5065, 0.4935], grad_fn=<SqueezeBackward1>)]

In [5]:
global_idx =actor.conditional_idx2global_idx(1,10, 5, 3, p_del[0])

In [6]:
global_idx

121

In [7]:
actor.global_idx2conditional_idx( global_idx, p_del[0])

(1, 0, 5, 3)

In [8]:
def conditional_to_global(add_idx,arm_idx,del_idx,p_act,p_add,p_arm,p_del,p_act_global):
    p_add_cond = (p_act[1]*p_add[add_idx]*p_arm[add_idx,arm_idx]).item()
    p_del_cond  = p_act[0]*p_del[del_idx].item()
    
    add_arm_glob_idx = actor.conditional_idx2global_idx(1,0,add_idx,arm_idx,p_del)
    global_del_idx = actor.conditional_idx2global_idx(0,del_idx,0,0,p_del)
    p_add_global =  p_act_global[add_arm_glob_idx].item()
    p_del_global =  p_act_global[global_del_idx].item()
    if p_add_cond != p_add_global :
        print(p_add_cond-p_add_global)
    assert abs(p_add_cond -p_add_global)<0.00001 
    assert p_del_cond == p_del_global
    

In [9]:
n  = dgl.unbatch(graphs)[0].number_of_nodes()
e = dgl.unbatch(graphs)[0].number_of_edges()
n,e

(31, 68)

In [10]:
###### add 
for add_idx in range(n):
    for arm_idx in range(10):
        conditional_to_global(add_idx,arm_idx,0,p_act[0],p_add[0],p_arm[0],p_del[0],p_global[0])
        
        
###### del idx 

for del_idx in range(e):
    conditional_to_global(0,0,del_idx,p_act[0],p_add[0],p_arm[0],p_del[0],p_global[0])

2.3283064365386963e-10
2.3283064365386963e-10
-2.3283064365386963e-10
2.3283064365386963e-10
2.3283064365386963e-10
2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
2.3283064365386963e-10
-2.3283064365386963e-10
2.3283064365386963e-10
-2.3283064365386963e-10
2.3283064365386963e-10
2.3283064365386963e-10
-2.3283064365386963e-10
2.3283064365386963e-10
2.3283064365386963e-10
2.3283064365386963e-10
2.3283064365386963e-10
2.3283064365386963e-10
2.3283064365386963e-10
2.3283064365386963e-10
2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
2.3283064365386963e-10
-2.3283064365386963e-10
2.3283064365386963e-10
2.3283064365386963e-10
-2.3283064365386963e-10
2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365386963e-10
-2.3283064365

In [11]:
state = dgl.unbatch(graphs)[0]

In [12]:
critic = Critic(config)

In [13]:
actor_critic = ActorCritic(actor,critic)

In [14]:
actor_critic.act(state)

(215,
 -5.8836846351623535,
 {'act': 1, 'del': 0, 'add': 14, 'arm': 7, 'global_act': 215})

In [15]:
torch.log(actor.get_prob(graphs)[2][0][107])

IndexError: index 107 is out of bounds for dimension 0 with size 68

In [15]:
torch.log(actor.get_prob(graphs)[-1][0][215])

tensor(-5.8837, grad_fn=<LogBackward>)

In [16]:
torch.log(actor.get_prob(graphs)[2][0][14]*actor.get_prob(graphs)[3][0][14,7]*actor.get_prob(graphs)[0][0][1])

tensor(-5.8837, grad_fn=<LogBackward>)

In [20]:
action = torch.LongTensor([8,10,20,108,2,67,19,19,11,23,67,89,109])

In [24]:
actor_critic.evaluate(graphs,action)

(tensor([ -4.0807, -15.9424, -15.9424, -15.9424,  -4.3162,  -5.8176,  -3.1657,
         -15.9424, -15.9424,  -4.3689, -15.9424, -15.9424, -15.9424],
        grad_fn=<SqueezeBackward1>),
 tensor([[-0.0515],
         [-0.0523],
         [-0.0517],
         [-0.0522],
         [-0.0513],
         [-0.0523],
         [-0.0519],
         [-0.0567],
         [-0.0519],
         [-0.0514],
         [-0.0514],
         [-0.0508],
         [-0.0515]], grad_fn=<AddmmBackward>),
 tensor([5.0028, 4.0781, 4.8137, 4.4974, 5.1479, 4.3920, 4.3884, 4.1299, 4.4223,
         5.2208, 5.0952, 5.1183, 4.7029], grad_fn=<NegBackward>))

In [33]:
(actor.get_prob(graphs)[-1][9]*torch.log(actor.get_prob(graphs)[-1][9]+1e-10)).sum()

tensor(-5.2208, grad_fn=<SumBackward0>)